Conduct Multiple Experiments ChipWhisperer into Custom File Format

In [ ]:
from WPI_SCA_LIBRARY.CWScope import *

# initialize scope
scope = CWScope("../WPI_SCA_LIBRARY/firmware/simpleserial-aes-CWLITEARM.hex")

# capture 1000 traces and store in an experiment named CWCapture1
scope.cw_to_file_format(1000, file_name="TestFile", experiment_name="CWCapture1", file_existing=False)

# capture 1500 traces and store in an experiment named CWCapture2
scope.cw_to_file_format(1500, file_name="TestFile", experiment_name="CWCapture2", file_existing=True)

Perform All Metrics on Data from the File

General File Format Demo